In [7]:
import numpy as np
from copy import deepcopy
import math
import time
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


n = 0
m = 0

def read_file():
    first_line = []
    lines_after_1 = []
    global n
    global m
    
    with open('testOsnovniSimpex.txt') as f:
        first_line = f.readline().strip('\n')
    n,m = first_line.split(" ")
    n = int(n)
    m = int(m)
    with open('testOsnovniSimpex.txt') as f:
        lines_after_1 = f.readlines()[1:]
    
    new_list = [item.strip() for item in lines_after_1]
    new_list = [i for i in new_list if i]
    
    matrix = [[float(num) for num in line.split(' ')] for line in new_list]
    coefficients = np.array(matrix)
    
    c_coefficient = np.array(matrix[n+m+1])
    b_coefficient = np.array(matrix[n+m])

    matrix2 = deepcopy(coefficients)
    matrix2 = np.delete(matrix2, n+m+1, 0)
    matrix2 = np.delete(matrix2, n+m, 0)
    print(n)
    return [matrix2, b_coefficient, c_coefficient]

def initialize_simplex(A, b, c):
    if min(b) >= 0:
        N = np.arange(1, n + 1, 1)
        B = np.arange(1+ n, n + m + 1, 1)
        v = 0
        return [A,b,c,N,B,v]
    else:
        return "Error! Negative Value Detected!"

def check_if_k_element_of_N(c, N):

    flag = False
    flag_location = 0
    for i in range(len(N)):
        if c[N[i] - 1] > 0:
            flag_location = N[i] - 1
            flag = True
            break
        else:
            flag = False
    return flag

def find_index_e(c,N):
    flag = False
    flag_location = 0
    for i in range(len(N)):
        if c[N[i] - 1] > 0:
            flag_location = N[i] - 1
            flag = True
            break
        else:
            flag = False
    return flag_location
        
def pivot(A, b, c, N, B, v, l, e):
    n_a = np.zeros((n+m, n+m))
    n_b = np.zeros(n+m)
    n_c = np.zeros(n+m)

    print(n_b)
    n_b[e] = b[l] / A[l][e]
    print(n_b)
    for j in range(len(N)):
        if(N[j] != e + 1):
            n_a[e][N[j] - 1] = A[l][N[j] - 1] / A[l][e]
        else:
            continue
            
    n_a[e][l] = 1 / A[l][e];
    
    for i in range(len(B)):
        if B[i] != l + 1:
            n_b[B[i] - 1] = b[B[i] - 1] - A[B[i] - 1][e] * n_b[e]
            for j in range (len(N)):
                if N[j] != e + 1:
                    n_a[B[i] - 1][N[j] - 1] = A[B[i] - 1][N[j] - 1] - A[B[i] - 1][e] * n_a[e][N[j]-1]
                else:
                    continue
            n_a[B[i] - 1][l] = -A[B[i] - 1][e] * n_a[e][l]
        else:
            continue
    n_v = v + c[e] * n_b[e]
    
    for j in range (len(N)):
        if(N[j] != e + 1):
            n_c[N[j] - 1] = c[N[j] - 1] - c[e] * n_a[e][N[j] - 1]
        else:
            continue
    n_c[l] = -c[e] * n_a[e][l]


    arr_N = list(N); 
    index_of_e_in_N = arr_N.index(e + 1)
    arr_N.remove(e + 1);

    arr_B = list(B)
    index_of_l_in_B = arr_B.index(l + 1)

    arr_B.remove(l + 1)
    
    arr_N.insert(index_of_e_in_N,l + 1)
    arr_B.insert(index_of_l_in_B,e + 1)

  
    n_N = arr_N
    n_B = arr_B

    n_N.sort()
    n_B.sort()
    

    
    return [n_a, n_b, n_c, n_N, n_B, n_v]     
            
def simplex(A,b,c):
    A, b, c, N, B, v = initialize_simplex(A, b, c)
    delta_j = []
    x = []
    counter = 0
    while (check_if_k_element_of_N(c,N)):
        counter = counter + 1
            
        delta_j = []
        _e = find_index_e(c,N)
        for j in range(len(B)):
            if A[B[j] - 1][_e] > 0:
                j_result = b[B[j]-1] / ( A[B[j] - 1][_e])


                delta_j.append(j_result)
            else:
                delta_j.append(math.inf)
        _l = np.argmin(delta_j)    

        if(delta_j[_l] == math.inf):
            return "Error! Neomejen Program"
        else:

            A, b, c , N, B, v = pivot(A, b, c, N, B, v, B[_l] - 1, _e )


    sumA_B = len(A) + len(B)
    i = 1
    for i in range(sumA_B):
        if( i in B):
            x.append(b[i - 1])
        else:
            x.append(0)
    z = v
    
    return [x,z]
        

def test_simplex():
    xArray = []
    yArray = []
    
    yArrayTemp = []
    yArrayMin = []
    yArrayMid = []
    yArrayMax = []
    
    for n in range(3, 100):
        yArrayTemp =[]
        for k in range(1,50):
            A = np.random.randint(low=0, high=9024, size=(n, n))
            b = np.random.randint(low=0, high=9024, size=n)
            c = np.random.randint(low=0, high=9024, size=n)
            start_time = time.perf_counter_ns()
            simplex(A,b,c)
            elapsed_time = time.perf_counter_ns() - start_time
            yArrayTemp.append(elapsed_time)
        
        xArray.append(n)
        np.sort(yArrayTemp)
        mid_index = int((len(yArrayTemp) - 1)/2)
        yArrayMin.append(min(yArrayTemp))
        yArrayMid.append(yArrayTemp[mid_index])
        yArrayMax.append(max(yArrayTemp))
        #print(yArrayTemp)
        #print("Min: ", min(yArrayTemp))
        #print("Mid: ", yArrayTemp[mid_index])
        #print("Max: ", max(yArrayTemp))

 
    sns.regplot(xArray, yArrayMin)
    plt.scatter(xArray,yArrayMin)
    plt.show()
    sns.regplot(xArray, yArrayMid)

    plt.scatter(xArray,yArrayMid)
    plt.show()
    sns.regplot(xArray, yArrayMax)

    plt.scatter(xArray,yArrayMax)
    plt.show()

A,b,c= read_file()
simplex(A, b, c)
test_simplex()

3
[0. 0. 0. 0. 0. 0.]
[55.  0.  0.  0.  0.  0.]
[0. 0. 0. 0. 0. 0.]
[ 0. 45.  0.  0.  0.  0.]
[0. 0. 0. 0. 0. 0.]
[ 0.  0. 18.  0.  0.  0.]
[0. 0. 0. 0. 0. 0.]
[ 0.  0.  0.  0.  0. 40.]


IndexError: index 3 is out of bounds for axis 0 with size 3